In [59]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
import unicodedata
import re

Team name constants

In [60]:
team_tags_dico = {
    "SCL" : ["scl","scald"],
    "IWG" : ["iwg"]
}

Instanciate Selenium web browser

In [61]:
driver_options = webdriver.FirefoxOptions()
driver_options.add_argument("--headless")
driver = webdriver.Firefox(options=driver_options)

In [62]:
def getBlue_picks_bans(soup,name):
    picks = soup.find(attrs={"class": "roomPickColumn blue"}).get_text("|")
    picks = picks.split("|")
    bans = [x['alt'] for x in soup.find(attrs={"class": "roomBanRow blue"}).find_all('img')]
    return picks,bans, name

In [63]:
def getRed_picks_bans(soup,name):
    picks = soup.find(attrs={"class": "roomPickColumn red"}).get_text("|")
    picks = picks.split("|")
    bans = [x['alt'] for x in soup.find(attrs={"class": "roomBanRow red"}).find_all('img')]
    return picks, bans, name

In [64]:
def detect_team_in_name(name,team_reference_dict, side) :

    for tag, keywords in team_reference_dict.items():
        for  word in keywords :
            if word in name.lower()  :
                return tag
    return name

In [65]:
def extract_date(name):

    match = re.search(r'\d', name)
    if match :
        digit_index = match.start()
        return name[digit_index:]
    return "NaT"

In [66]:
def get_side_by_tag(soup,team_reference_dict):

    blue_text = unicodedata.normalize("NFKD",soup.find(attrs={"class" : "roomReadyBackground roomReadyBackgroundblue"}).previous_sibling.get_text())
    red_text = unicodedata.normalize("NFKD",soup.find(attrs={"class" : "roomReadyBackground roomReadyBackgroundred"}).previous_sibling.get_text())
    
    blue_team = detect_team_in_name(blue_text, team_reference_dict, "Blue")
    red_team = detect_team_in_name(red_text,team_reference_dict, "Red")

    if blue_team == "SCL" :
        game_date = extract_date(blue_text)
    elif red_team == "SCL" :
        game_date = extract_date(red_text)
    else :
        game_date = "NaT"
    return blue_team, red_team, game_date

In [67]:
def scraping_draft(draft_url,team_reference_dict):

    driver.get(draft_url)
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    
    teams_names = get_side_by_tag(soup,team_reference_dict)

    blue = getBlue_picks_bans(soup,teams_names[0])
    red = getRed_picks_bans(soup,teams_names[1])

    bdd_draft_df = pd.DataFrame(columns=['link','draft_picks','draft_bans','team','side','date'])
    bdd_draft_df = pd.concat([bdd_draft_df, pd.DataFrame([[draft_url,blue[0],blue[1],blue[2],'b',teams_names[2]]],columns=bdd_draft_df.columns)],axis=0)
    
    
    return bdd_draft_df

In [ ]:
scraping_draft("https://draftlol.dawe.gg/ChTA4fbR",team_tags_dico)